In [1]:
#Importing Libraries
import os
import numpy as np 

import tensorflow as tf 
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten

In [2]:
#Fetch Images count from folders

count = 0
dirs = os.listdir('flowers/')

for folder in dirs:
    path = os.path.join('Flowers', folder)
    if os.path.isdir(path):
        files = os.listdir(path)
        print(f"{folder} Folder has {len(files)} Images")
        count += len(files)

print(f"Total Images in Flowers Folder: {count}")

daisy Folder has 764 Images
dandelion Folder has 1052 Images
rose Folder has 784 Images
sunflower Folder has 733 Images
tulip Folder has 984 Images
Total Images in Flowers Folder: 4317


In [3]:
#Load Images into Arrays as Dataset

base_dir = 'flowers/'
img_size = 180
batch = 3

train_dataset = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed = 123,
    validation_split = 0.2,
    subset = 'training',
    batch_size = batch,
    image_size = (img_size, img_size)
)

validatin_dataset = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed = 123,
    validation_split = 0.2,
    subset = 'validation',
    batch_size = batch,
    image_size = (img_size, img_size)
)

Found 4317 files belonging to 5 classes.
Using 3454 files for training.
Found 4317 files belonging to 5 classes.
Using 863 files for validation.
